In [1]:
load('../sim_data/toe_Elnet.RData')

In [2]:
dir = '../sim_data'
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

rou.list = seq(0.1, 0.9, 0.2)

files = NULL
for (rou in rou.list){
  for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_toeplitz_corr', rou, paste('P', p, 'N', n, sep='_'), '.RData', sep=''))
  }
}

In [3]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_toe_elnet[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}

In [4]:
table_toe = NULL
tmp_num_select = rep(0, length(results_toe_elnet))
for (i in 1:length(results_toe_elnet)){
    table_toe = rbind(table_toe, results_toe_elnet[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_toe_elnet[[i]]$Stab.table))
    
}
table_toe = as.data.frame(table_toe)
table_toe$num_select = tmp_num_select
table_toe$FDR = round(avg_FDR,2)

In [5]:
head(table_toe)

n,p,rou,FP,FN,MSE,Stab,num_select,FDR
50,50,0.1,14.72 ( 0.57 ),0.01 ( 0.01 ),0.6 ( 0.03 ),0.21,19.71,0.67
100,50,0.1,13.7 ( 0.66 ),0 ( 0 ),0.34 ( 0.01 ),0.23,18.70,0.64
500,50,0.1,13.59 ( 0.55 ),0 ( 0 ),0.26 ( 0 ),0.23,18.59,0.65
1000,50,0.1,12.64 ( 0.54 ),0 ( 0 ),0.26 ( 0 ),0.25,17.64,0.63
50,100,0.1,19.33 ( 0.67 ),0 ( 0 ),0.69 ( 0.05 ),0.2,24.33,0.73
100,100,0.1,17.77 ( 0.8 ),0 ( 0 ),0.38 ( 0.01 ),0.22,22.77,0.70


In [6]:
# export result
result.table_toe <- apply(table_toe,2,as.character)
rownames(result.table_toe) = rownames(table_toe)
result.table_toe = as.data.frame(result.table_toe)

# extract numbers only for 'n' & 'p'
result.table_toe$n = tidyr::extract_numeric(result.table_toe$n)
result.table_toe$p = tidyr::extract_numeric(result.table_toe$p)
result.table_toe$ratio = result.table_toe$p / result.table_toe$n

result.table_toe = result.table_toe[c('n', 'p', 'rou', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_toe)[1:4] = c('N', 'P', 'Corr', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [7]:
# convert interested measurements to be numeric
result.table_toe$Stab = as.numeric(as.character(result.table_toe$Stab))
result.table_toe$MSE_mean = as.numeric(substr(result.table_toe$MSE, start=1, stop=4))
result.table_toe$FP_mean = as.numeric(substr(result.table_toe$FP, start=1, stop=4))
result.table_toe$FN_mean = as.numeric(substr(result.table_toe$FN, start=1, stop=4))
result.table_toe$FN_mean[is.na(result.table_toe$FN_mean)] = 0
result.table_toe$num_select = as.numeric(as.character(result.table_toe$num_select))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [8]:
# check whether missing values exists
result.table_toe[rowSums(is.na(result.table_toe)) > 0,]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
24,1000,100,0.3,0.1,0.20,0.26 ( 0 ),19 ( 0.86 ),0 ( 0 ),24.00,0.71,0.26,NA,0.00
66,100,50,0.9,0.5,0.21,0.35 ( 0.01 ),14 ( 0.69 ),0.27 ( 0.06 ),18.73,0.66,0.35,NA,0.27


In [9]:
# recover values
result.table_toe$FP_mean[24] = 19
result.table_toe$FP_mean[66] = 14

In [10]:
result.table_toe[c(24, 66), ]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
24,1000,100,0.3,0.1,0.20,0.26 ( 0 ),19 ( 0.86 ),0 ( 0 ),24.00,0.71,0.26,19,0.00
66,100,50,0.9,0.5,0.21,0.35 ( 0.01 ),14 ( 0.69 ),0.27 ( 0.06 ),18.73,0.66,0.35,14,0.27


In [11]:
head(result.table_toe)

N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
50,50,0.1,1.00,0.21,0.6 ( 0.03 ),14.72 ( 0.57 ),0.01 ( 0.01 ),19.71,0.67,0.60,14.7,0.01
100,50,0.1,0.50,0.23,0.34 ( 0.01 ),13.7 ( 0.66 ),0 ( 0 ),18.70,0.64,0.34,13.7,0.00
500,50,0.1,0.10,0.23,0.26 ( 0 ),13.59 ( 0.55 ),0 ( 0 ),18.59,0.65,0.26,13.5,0.00
1000,50,0.1,0.05,0.25,0.26 ( 0 ),12.64 ( 0.54 ),0 ( 0 ),17.64,0.63,0.26,12.6,0.00
50,100,0.1,2.00,0.20,0.69 ( 0.05 ),19.33 ( 0.67 ),0 ( 0 ),24.33,0.73,0.69,19.3,0.00
100,100,0.1,1.00,0.22,0.38 ( 0.01 ),17.77 ( 0.8 ),0 ( 0 ),22.77,0.7,0.38,17.7,0.00


In [12]:
tail(result.table_toe)

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
75,500,500,0.9,1.0,0.12,0.29 ( 0 ),41.14 ( 1.02 ),0 ( 0 ),46.14,0.86,0.29,41.1,0.00
76,1000,500,0.9,0.5,0.13,0.27 ( 0 ),39.21 ( 0.84 ),0 ( 0 ),44.21,0.86,0.27,39.2,0.00
77,50,1000,0.9,20.0,0.04,0.94 ( 0.05 ),36.59 ( 3.6 ),3.72 ( 0.08 ),37.87,0.92,0.94,36.5,3.72
78,100,1000,0.9,10.0,0.09,0.59 ( 0.02 ),32.05 ( 1.85 ),2.59 ( 0.06 ),34.46,0.88,0.59,32.0,2.59
79,500,1000,0.9,2.0,0.09,0.3 ( 0 ),57.69 ( 1.57 ),0 ( 0 ),62.69,0.9,0.30,57.6,0.00
80,1000,1000,0.9,1.0,0.10,0.27 ( 0 ),52.35 ( 1.6 ),0 ( 0 ),57.35,0.89,0.27,52.3,0.00


In [13]:
## export
write.table(result.table_toe, '../results_summary/sim_toe_elnet.txt', sep='\t', row.names=F)

In [10]:
library(ggplot2)
library(ggpubr)
result.table_toe$N = as.factor(result.table_toe$N)
fig_toe_stab = ggplot(result.table_toe, aes(x=P, y=Stab, color=N)) + 
               geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
               scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('Stability')

fig_toe_mse = ggplot(result.table_toe, aes(x=P, y=MSE_mean, color=N)) + 
              geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position="none") +
              scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('MSE')

fig_toe_fp = ggplot(result.table_toe, aes(x=P, y=FP_mean, color=N)) + 
             geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
             scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('False Positives')

fig_toe_fn = ggplot(result.table_toe, aes(x=P, y=FN_mean, color=N)) + 
             geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
             scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('False Negatives')
fig = ggarrange(fig_toe_stab, fig_toe_mse, fig_toe_fp, fig_toe_fn, ncol=2, nrow=2, 
          common.legend = TRUE, legend="right") 
fig = annotate_figure(fig, top = text_grob("Toeplitz_ElaticNet"))
ggexport(fig, filename = "../figures_sim/figure_toe_elnet.pdf", height=6, width=6)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: magrittr
Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”file saved to ../figures_sim/figure_toe_elnet.pdf


In [11]:
result.table_toe[with(result.table_toe, order(N, P, Corr)),]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,MSE_mean,FP_mean,FN_mean
1,50,50,0.1,1.0,0.21,0.6 ( 0.03 ),14.72 ( 0.57 ),0.01 ( 0.01 ),0.60,14.7,0.01
17,50,50,0.3,1.0,0.21,0.53 ( 0.03 ),14.65 ( 0.58 ),0.03 ( 0.02 ),0.53,14.6,0.03
33,50,50,0.5,1.0,0.20,0.57 ( 0.04 ),14.64 ( 0.66 ),0.09 ( 0.03 ),0.57,14.6,0.09
49,50,50,0.7,1.0,0.18,0.6 ( 0.04 ),15.25 ( 0.54 ),0.25 ( 0.05 ),0.60,15.2,0.25
65,50,50,0.9,1.0,0.16,0.5 ( 0.02 ),13.26 ( 0.77 ),1.03 ( 0.11 ),0.50,13.2,1.03
5,50,100,0.1,2.0,0.20,0.69 ( 0.05 ),19.33 ( 0.67 ),0 ( 0 ),0.69,19.3,0.00
21,50,100,0.3,2.0,0.19,0.72 ( 0.04 ),19.67 ( 0.66 ),0.08 ( 0.03 ),0.72,19.6,0.08
37,50,100,0.5,2.0,0.17,0.8 ( 0.04 ),20.05 ( 0.68 ),0.34 ( 0.06 ),0.80,20.0,0.34
53,50,100,0.7,2.0,0.15,0.81 ( 0.04 ),19.25 ( 0.81 ),0.79 ( 0.09 ),0.81,19.2,0.79
69,50,100,0.9,2.0,0.15,0.56 ( 0.03 ),15.35 ( 0.8 ),1.83 ( 0.12 ),0.56,15.3,1.83
